# Subword Tokenizer

깃 설명 확인 (표)

### 네이버 영화리뷰 학습 

In [ ]:
# 감정분석 때 사용 할테니깐 잘 가지고 있기

In [1]:
# 네이버 영화 리뷰 데이터 로드
import urllib.request
import os

# 파일 다운로드 함수 
def get_file(filname, origin):
    cache_dir = os.path.expanduser('~/.torch/datasets')  # 캐쉬 폴더 잡기 
    os.makedirs(cache_dir, exist_ok=True)                # 생성 (exist_ok : 존재하는 경우 오류어쩌구 이거 뭐냐  )
    filepath = os.path.join(cache_dir, filname)          # 파일 경로 만들기 

    if not os.path.exists(filepath):                     # 파일이 존재하는 경우에만 다운로드 
        print(f'Download 진행중 {origin}')
        urllib.request.urlretrieve(origin, filepath)

    return filepath                                      # 경로 반환

In [ ]:
ratigs_train_path = get_file('ratings_train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt') # 파일 이름, url 입력 > 파일 경로 반환
ratigs_test_path = get_file('ratings_test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')    # 테스트 데이터 반환

ratigs_test_path, ratigs_train_path   # label = 감정분석 데이터이기 때문에 긍정, 부정 의미


Download 진행중 https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Download 진행중 https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt


('C:\\Users\\Playdata/.torch/datasets\\ratings_test.txt',
 'C:\\Users\\Playdata/.torch/datasets\\ratings_train.txt')

In [ ]:
import pandas as pd 

ratigs_train_df = pd.read_csv(ratigs_train_path, sep='\t')  # \t = teb의미 (원본 데이터가 구분되어있는 형태 정의해줌)
ratigs_test_df = pd.read_csv(ratigs_test_path, sep='\t')

display(ratigs_test_df)
display(ratigs_train_df)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
ratigs_test_df.isna().sum() # 결측치 확인

id          0
document    3
label       0
dtype: int64

In [6]:
# 결측치 제거 
ratigs_train_df = ratigs_train_df.dropna(how='any')
ratigs_test_df = ratigs_test_df.dropna(how='any')

ratigs_train_df.shape, ratigs_test_df.shape

((149995, 3), (49997, 3))

In [ ]:
with open('naver_review.txt', 'w', encoding='utf-8') as f:
    for doc in ratigs_train_df['document'].values:  # document 값만 기준으로 개행해서 txt로 저장
        f.write(doc + '\n')

### SentencePieceTokenizer

In [ ]:
# !pip install sentencepiece

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 25.6 MB/s  0:00:00


In [ ]:
import sentencepiece as spt 

input = 'naver_review.txt'
vocab_size = 10000
model_prefix = 'naver_review'
cmd = f'--input={input} --model_prefix={model_prefix} --vocab_size={vocab_size}'  # 옵션에 대한 내용 

spt.SentencePieceTrainer.Train(cmd)  # 같은 경로에 모델과 vocab 반환 (각 단어들만 반환하게 훈련된 모델)

In [ ]:
sp = spt.SentencePieceProcessor()
sp.Load(f'{model_prefix}.model')  # 학습한 모델 로드 

for doc in ratigs_train_df['document'].values[:3]: # 3개 문장에 대한 
    print(doc)                                     # 원본                                   # 아 더빙.. 진짜 짜증나네요 목소리
    print(sp.encode_as_pieces(doc))                # 토큰화 결과 (앞 공백도 문자로 인식)                 # ['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
    print(sp.encode_as_ids(doc))                   # vocab에서의 인덱스 번호 반환 (subword tokenizer 후에)       # [62, 877, 5, 31, 2019, 68, 1710]
    print()

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
[62, 877, 5, 31, 2019, 68, 1710]

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터', '보고', '▁초딩', '영화', '줄', '....', '오', '버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1634, 8, 4908, 159, 1460, 33, 264, 60, 173, 548, 410, 1224, 7396, 754, 440]

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보', '는것을', '추천', '한다']
[23, 369, 9781, 429, 3780, 143, 6266, 1945, 314]



In [ ]:
# 단어사전 / 모델의 어휘 크기 (vocabulary size 출력)
sp.get_piece_size() # 10000 = vocab 사이즈 
sp.GetPieceSize()

10000

In [25]:
# 인코딩
text = ratigs_test_df['document'][100]
tokens = sp.encode_as_pieces(text) # 텍스트 -> subword 단위 분할
id_tokens = sp.encode_as_ids(text) # 텍스트 -> subword 단위 분할 -> 고유 ID로 변환

print(text)
print(tokens)
print(id_tokens)

# 직접 디코딩
print("".join(tokens).replace("▁", " ").strip()) # ▁ 공백 처리 


# 디코딩 (함수 사용)
print(sp.decode_pieces(tokens))
print(sp.decode_ids(id_tokens))

걸작은 몇안되고 졸작들만 넘쳐난다.
['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
[1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.


### BertWordPieceTokenizer

In [ ]:
# !pip install tokenizers

In [18]:
from tokenizers import BertWordPieceTokenizer

# lowercase = False (소문자로 토큰관리 x / 한국어에는 대소문자 없음)
# strip_accents = False (발음 강세문자 기호 제거 x / 한국어에서는 자모 분리 가능성 있을 유)
tokenizer = BertWordPieceTokenizer(lowercase = False, strip_accents = False)  
vocab_size = 10000

tokenizer.train(
    files = ['naver_review.txt'],  # 토큰화할때 사용할 데이터 
    vocab_size = vocab_size,       
    min_frequency = 5,             # 확률기반 계산하는 친구이기 때문에 최소 5번 이상 등장한 친구만 vocab에 포함
    show_progress = True
)

In [ ]:
tokenizer.save_model('./', 'bert_word_piece_from_naver_review')  # 모델 저장하는 방법 (알아만둬)

['./bert_word_piece_from_naver_review-vocab.txt']

In [24]:
text = ratigs_test_df['document'][100]
encoded = tokenizer.encode(text)

print(encoded.tokens)   # encoded 반환시 객체로 반환하기 때문에 토큰화된 걸 보려면 tokens를 붙여야함 
print(encoded.ids)

print(tokenizer.decode(encoded.ids))

['걸작', '##은', '몇', '##안되고', '졸작', '##들만', '넘쳐', '##난다', '.']
[2759, 1115, 445, 9508, 2589, 3799, 8335, 2430, 16]
걸작은 몇안되고 졸작들만 넘쳐난다.


- sentencepiece 결과 
걸작은 몇안되고 졸작들만 넘쳐난다.
['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
[1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]

In [ ]:
# 차이가 있는 것을 확인할 수 있음. 